<a href="https://colab.research.google.com/github/ransjnr/building-ai-agents-with-autogen/blob/main/coding_copilot_with_autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autogen
!pip install yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.5/787.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.5 MB/s eta 0:00:00


In [ ]:
import autogen
import os
import openai
from openai import AzureOpenAI
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import AssistantAgent, ConversableAgent
from google.colab import userdata

In [ ]:
endpoint = userdata.get("AZURE_OPENAI_4O_ENDPOINT")
model_name = "o4-mini"
deployment = "o4-mini-deployment"

subscription_key = userdata.get("AZURE_OPENAI_4O_KEY")
api_version = "2024-12-01-preview"

In [ ]:
llm_config = {
    "config_list": [
        {
            "model": deployment,
            "base_url": endpoint,
            "api_type": "azure",
            "api_key": subscription_key,
            "api_version": api_version,
        }
    ]
}

###Create an executor instance

In [ ]:
executor = LocalCommandLineCodeExecutor(
    timeout=10,
    work_dir="coding",
)

###Define Agents

In [ ]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply =
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

In [ ]:
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [ ]:
code_writer_agent_system_message = code_writer_agent.system_message

In [ ]:
code_writer_agent_system_message

'You are a helpful AI assistant.\nSolve tasks using your coding and language skills.\nIn the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.\n    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.\n    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.\nSolve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.\nWhen using code, you must indicate the script type in the code block. The user cannot provide any oth

###Defining stock analysis task

In [ ]:
from datetime import date

current_date = date.today()

instruction = f"""
As of {current_date}, please generate a graph displaying the year-to-date (YTD) stock performance for NVIDIA (NVDA) and Tesla (TSLA).

Ensure that you:
1. Write the necessary code within a markdown code block
2. Save the resulting graph as an image file named 'ytd_stock_gains.png'
"""

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=instruction,
)

code_executor_agent (to code_writer_agent):


As of 2025-05-13, please generate a graph displaying the year-to-date (YTD) stock performance for NVIDIA (NVDA) and Tesla (TSLA).

Ensure that you:
1. Write the necessary code within a markdown code block
2. Save the resulting graph as an image file named 'ytd_stock_gains.png'


--------------------------------------------------------------------------------
[autogen.oai.client: 05-13 21:52:35] {706} WARNING - Model o4-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


code_writer_agent (to code_executor_agent):

Here’s a self-contained Python script that pulls the 2025 YTD data for NVDA and TSLA, computes their year-to-date gains, plots them, and saves the figure as **ytd_stock_gains.png**.

```python
# filename: plot_ytd_stock.py

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

def main():
    # Define symbols and date range
    symbols = ['NVDA', 'TSLA']
    start_date = '2025-01-01'
    end_date = '2025-05-13'

    # Download adjusted close prices
    data = yf.download(symbols, start=start_date, end=end_date)['Adj Close']

    # Compute YTD % gain relative to first trading day
    pct_gain = (data.div(data.iloc[0]) - 1) * 100

    # Plot
    plt.figure(figsize=(10, 6))
    for sym in symbols:
        plt.plot(pct_gain.index, pct_gain[sym], label=f"{sym}")
    plt.title("2025 YTD Stock Performance: NVDA vs TSLA")
    plt.xlabel("Date")
    plt.ylabel("YTD Gain (%)")
    plt.legend()
    plt.grid(True)
    plt.tight_layo

In [ ]:
import os
from IPython.display import Image

Image(os.path.join("coding", "ytd_stock_gains.png"))

FileNotFoundError: No such file or directory: 'coding/ytd_stock_gains.png'

FileNotFoundError: No such file or directory: 'coding/ytd_stock_gains.png'

<IPython.core.display.Image object>

##Custom functions for stock analysis

###Functions

In [ ]:
## function to get stock data
def get_stock_prices(
    stock_symbols,
    start_date,
    end_date,
):
  """
  Fetch stock closing prices for given stock symbols between start and end dates.

  Args:
    stock_symbols: Stock symbols(s) to fetch prices for. Can be a string or a list of strings.
    start_datae: The start date in 'YYYY-MM-DD' format.
    end_date: The end date in 'YYYY_MM_DD' format.

  Returns:
    DataFrame: Stock closing prices indexed by date, with one column per stock symbol.
  """
  import yfinance
  from pandas import DataFrame

  try:
    stock_data=yfinance.download(stock_symbols, start=start_date, end=end_date)
    return stock_data['Close']
  except Exception as e:
    print(f"Error fetching stock data: {str(e)}")
    return DataFrame()

In [ ]:
##plot the stock prices
import matplotlib.pyplot as plt
from pandas import DataFrame

def plot_stock_prices(stock_prices, filename):
  """
  Plot stock prces and save the figure to a file.

  Args:
    stock_prices (DataFrame): Stock prices indexed by date, with columns for each stock.
    filename (str): Name of the file to save the plot.
  """
  plt.figurue(figsize=(10,5))
  stock_prices.plot(ax=plt.gca(),grid=True)
  plt.title("Stock Prices")
  plt.xlabel("Date")
  plt.ylabel("Price")
  plt.legend(loc='best')
  plt.savefig(filename)
  plt.close()

###A new executor with custom functions

In [ ]:
executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir="coding",
    functions=[get_stock_prices, plot_stock_prices],
)

In [ ]:
code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply "TERMINATE" in the end when everything is done.
"""

In [ ]:
code_writer_system_message += executor.format_functions_for_prompt()
print(code_writer_system_message)

You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or 

###Create agents with updated system message

In [ ]:
code_writer_agent = ConversableAgent(
    name="code_writer_agent",
    system_message=code_writer_system_message,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [ ]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply =
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

In [ ]:
from datetime import date

chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=(
        f"Today is {date.today()}. "
        "Download the stock prices YTD for NVDA and TSLA and create "
        "a plot. Make sure the code is in a markdown code block and "
        "save the figure to a file stock_prices_YTD_plot.png."
    )
)


code_executor_agent (to code_writer_agent):

Today is 2025-05-14. Download the stock prices YTD for NVDA and TSLA and create a plot. Make sure the code is in a markdown code block and save the figure to a file stock_prices_YTD_plot.png.

--------------------------------------------------------------------------------
[autogen.oai.client: 05-14 01:32:48] {706} WARNING - Model o4-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


code_writer_agent (to code_executor_agent):

```python
# filename: plot_stock_prices_ytd.py
from functions import get_stock_prices, plot_stock_prices
from datetime import date

def main():
    # Define the date range for Year-to-Date (YTD) up to 2025-05-14
    start_date = "2025-01-01"
    end_date = "2025-05-14"
    symbols = ["NVDA", "TSLA"]

    # Fetch closing prices
    stock_prices = get_stock_prices(symbols, start_date, end_date)

    # Create and save the plot
    output_file = "stock_prices_YTD_plot.png"
    plot_stock_prices(stock_prices, output_file)
    print(f"Plot saved to {output_file}")

if __name__ == "__main__":
    main()
```

Run this script with:
```sh
python plot_stock_prices_ytd.py
```
TERMINATE

--------------------------------------------------------------------------------
Replying as code_executor_agent. Provide feedback to code_writer_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>

code_writer_agent (to code_executor_agent):

Here’s a self‐contained script that bypasses the buggy `plot_stock_prices` and does the plotting directly with matplotlib. Save it as `plot_stock_prices_ytd.py` and run it.

```python
# filename: plot_stock_prices_ytd.py
from functions import get_stock_prices
import matplotlib.pyplot as plt

def main():
    # Define the date range for Year‐to‐Date (YTD) up to 2025-05-14
    start_date = "2025-01-01"
    end_date   = "2025-05-14"
    symbols    = ["NVDA", "TSLA"]

    # Fetch closing prices
    stock_prices = get_stock_prices(symbols, start_date, end_date)

    # Plot
    plt.figure(figsize=(10, 6))
    for symbol in stock_prices.columns:
        plt.plot(stock_prices.index, stock_prices[symbol], label=symbol)
    plt.title("YTD Closing Prices for NVDA and TSLA (2025)")
    plt.xlabel("Date")
    plt.ylabel("Closing Price (USD)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Save
    output_file = "stock_prices_YTD_plot.pn

code_writer_agent (to code_executor_agent):

The plot of YTD closing prices for NVDA and TSLA has been created and saved as `stock_prices_YTD_plot.png`.  
TERMINATE

--------------------------------------------------------------------------------
Replying as code_executor_agent. Provide feedback to code_writer_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit

>>>>>>>> TERMINATING RUN (55f2f9c0-fcd7-4404-ab1e-e3add29dc64b): User requested to end the conversation
